# Jupyter File Drag & Drop Widget - Demo

This notebook demonstrates how to use the IFrameDropWidget for drag-and-drop file uploads in JupyterLab.

## FileDrop: Compact Multi-File API

The `FileDrop` class provides a 1-line API for multiple drop zones with automatic global listener installation.

In [1]:
# One-line creation and display!
import sys
sys.path.insert(0, '..')  # Add parent directory to find the module
from jupyter_iframe_upload import FileDrop

fd = FileDrop("Dataset A", "Dataset B" , "Dataset C")
fd.ui

<IPython.core.display.Javascript object>

In [2]:
# Access loaded DataFrames
print("Loaded datasets:", list(fd.datasets.keys()))

# Access individual DataFrame
df_a = fd["Dataset A"]  # Returns DataFrame or None
if df_a is not None:
    print(f"Dataset A shape: {df_a.shape}")
    display(df_a.head())

Loaded datasets: []


In [3]:
# Dynamic operations - add/remove drop zones
fd.add("Dataset D")    # Add new drop zone
# fd.remove("Dataset A")  # Remove zone and clear its data

# All methods return self for chaining:
# fd.add("C").add("D").remove("A")

FileDrop(labels=['Dataset A', 'Dataset B', 'Dataset C', 'Dataset D'], loaded=[])

In [4]:
fd.remove('Dataset B')

FileDrop(labels=['Dataset A', 'Dataset C', 'Dataset D'], loaded=[])

## Embedding FileDrop in Containers

The `.ui` property returns a widget that can be embedded in ipywidgets containers like Accordion, Tab, VBox, etc.

In [5]:
# Embedding in Accordion
import ipywidgets as widgets
from IPython.display import display

# Create FileDrop instances
fd_csv = FileDrop("CSV File 1", "CSV File 2")
fd_excel = FileDrop("Excel File")

# Create Accordion with FileDrop widgets
accordion = widgets.Accordion(children=[fd_csv.ui, fd_excel.ui])
accordion.set_title(0, "CSV Files")
accordion.set_title(1, "Excel Files")

display(accordion)

Accordion(children=(VBox(children=(HBox(children=(VBox(children=(HTML(value="<h4 style='margin:0 0 5px 0'>CSV …

In [6]:
# Embedding in Tab
fd_train = FileDrop("Training Data")
fd_test = FileDrop("Test Data")

tab = widgets.Tab(children=[fd_train.ui, fd_test.ui])
tab.set_title(0, "Training")
tab.set_title(1, "Testing")

display(tab)

In [7]:
# Embedding in VBox with other widgets
fd_upload = FileDrop("Upload File")
btn = widgets.Button(description="Process Data", button_style="primary")
output = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()
        if fd_upload.datasets:
            print("Processing data from:", list(fd_upload.datasets.keys()))
        else:
            print("No file uploaded yet!")

btn.on_click(on_click)

vbox = widgets.VBox([fd_upload.ui, btn, output])
display(vbox)

### Step 1: Import and Install Global Listener

**IMPORTANT:** Run this cell first, before creating any widgets.

In [ ]:
# Create the widget with our callback
drop_widget = IFrameDropWidget(on_dataframe_ready=on_file_loaded)

# Display it
drop_widget.display()

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to find the module

from jupyter_iframe_upload import IFrameDropWidget

# Install the global JavaScript listener - MUST be called first!
IFrameDropWidget.install_global_listener()
print('Global listener installed!')

### Step 2: Create a Variable to Store Data

We'll store the loaded DataFrame in a variable so we can use it later.

In [ ]:
# Variable to store the loaded DataFrame
loaded_data = {'filename': None, 'df': None}

### Step 3: Define a Callback Function

This function will be called when a file is successfully loaded.

In [ ]:
def on_file_loaded(filename, df):
    """Callback when a file is dropped and parsed."""
    # Store the data
    loaded_data['filename'] = filename
    loaded_data['df'] = df
    
    # Print summary
    print(f'\n{"="*50}')
    print(f'Loaded: {filename}')
    print(f'Shape: {df.shape[0]} rows x {df.shape[1]} columns')
    print(f'{"="*50}')
    print('\nColumns:')
    for col in df.columns:
        print(f'  - {col} ({df[col].dtype})')
    print('\nPreview:')
    display(df.head())

### Step 4: Create and Display the Drop Widget

Drag and drop a CSV, XLSX, or XLSM file onto the drop zone below.

### Step 5: Work with the Loaded Data

After dropping a file, you can access the data in `loaded_data['df']`.

In [ ]:
# Access the loaded DataFrame
if loaded_data['df'] is not None:
    df = loaded_data['df']
    print(f"Working with: {loaded_data['filename']}")
    print(f"\nBasic statistics:")
    display(df.describe())
else:
    print("No data loaded yet. Drop a file in the widget above!")